In [28]:
# Importing necessary libraries
import pandas as pd
from tinydb import TinyDB, Query

# Initialize the TinyDB database|
db = TinyDB('db.json')

# Define the table
modules_table = db.table('modules')




ModuleNotFoundError: No module named 'google.cloud'

In [34]:
modules_table._read_table()

[{'name': 'PAM2401SCADJ',
  'type': ['Regulator', 'AdjustableRegulator', 'BoostRegulator'],
  'properties': {'power_in': {'voltage': {'min': '0.9V', 'max': '4.75V'}},
   'power_out': {'voltage': {'min': '2.5V', 'max': '5V'},
    'current': {'min': '0A', 'max': '3A'}},
   'switching_frequency': '1MHz'}},
 {'name': 'LM317ADJ',
  'type': ['Regulator', 'AdjustableRegulator', 'BoostRegulator'],
  'properties': {'power_in': {'voltage': {'min': '1.2V', 'max': '37V'}},
   'power_out': {'voltage': {'min': '1.25V', 'max': '37V'},
    'current': {'min': '0A', 'max': '1.5A'}}}},
 {'name': 'TPS54331',
  'type': ['BuckRegulator', 'Regulator', 'AdjustableRegulator'],
  'properties': {'power_in': {'voltage': {'min': '3.5V', 'max': '28V'}},
   'power_out': {'voltage': {'min': '0.8V', 'max': '25V'},
    'current': {'min': '0A', 'max': '3A'}},
   'switching_frequency': '570kHz'}}]

In [15]:
import json

# Read the JSON file
with open('db.json') as f:
    data = json.load(f)

# stick it into a dataframe
df = pd.DataFrame(data['modules'])
df

JSONDecodeError: Expecting property name enclosed in double quotes: line 26 column 7 (char 577)

In [ ]:
def query_modules(selected_types, properties_filters):
    Module = Query()
    results = []

    # Search through all modules
    for module in modules_table.all():
        # Check if the module's type intersects with the selected types
        if set(module['type']).intersection(selected_types):
            # Further filter the module based on properties
            match = True
            for prop, value_range in properties_filters.items():
                # Assume each property filter defines min and max
                if 'min' in value_range and 'max' in value_range:
                    prop_min = float(module['properties'][prop]['min'].rstrip('V').rstrip('A'))  # Remove units for comparison
                    prop_max = float(module['properties'][prop]['max'].rstrip('V').rstrip('A'))  # Remove units for comparison
                    filter_min = float(value_range['min'])
                    filter_max = float(value_range['max'])
                    if not (filter_min <= prop_min and prop_max <= filter_max):
                        match = False
                        break
            if match:
                results.append(module)

    return results

# Sample types and properties_filters for testing
selected_types = ["Regulator", "AdjustableRegulator"]
properties_filters = {
    "power_in.voltage": {'min': '0.5', 'max': '5'},
    "power_out.voltage": {'min': '1', 'max': '6'}
}

# Query the database
results = query_modules(selected_types, properties_filters)

# Convert the results to a Pandas DataFrame for better display
df = pd.DataFrame(results)

df.head()  # Display the first few rows of the dataframe